In this project, I use Keras to conduct deep learning. Batch size, training epoch, learning rate, activation function, network weight initialisation and number of neurons in the layers are tuned using GridSearchCV. The best parameters obtained from tuning are used in the final model for prediction and model evaluation.

Import libraries.

In [127]:
import pandas as pd
import numpy as np
import sklearn
import keras

Read the dataset and assign column names.

In [117]:
names = ['n_pregnant', 'glucose_concentration', 'blood_pressure (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
df = pd.read_csv("pimaindiansdiabetes.csv",names=names)

In [118]:
df.head()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
n_pregnant                 768 non-null int64
glucose_concentration      768 non-null int64
blood_pressure (mm Hg)     768 non-null int64
skin_thickness (mm)        768 non-null int64
serum_insulin (mu U/ml)    768 non-null int64
BMI                        768 non-null float64
pedigree_function          768 non-null float64
age                        768 non-null int64
class                      768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


Glucose concentration, blood pressure, skin thickness, serum insulin and BMI have zero values which should not be the case.

In [120]:
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Replace zero values with NA values.

In [121]:
columns = ['glucose_concentration', 'blood_pressure (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI']

for col in columns:
    df[col].replace(0, np.NaN, inplace=True)

There are no more minimum values indicated as zero.

In [122]:
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Drop NA values. There are now 392 rows of data instead of initial 768 rows.

In [123]:
df.dropna(inplace=True)
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Get dimensions of the dataset.

In [124]:
dataset = df.values
print(dataset.shape)

(392, 9)


Separate columns into independent variables and target variable. The target variable is changed to integer type. Class 1 means person has diabetes. Class 0 means person does not have diabetes.

In [125]:
X = dataset[:,0:8]
Y = dataset[:, 8].astype(int)

Transform data into a normal distribution with mean 0 and standard deviation 1.

In [128]:
from sklearn.preprocessing import StandardScaler

In [129]:
scaler = StandardScaler().fit(X)
print(scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [130]:
X_standardised = scaler.transform(X)
data = pd.DataFrame(X_standardised)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-9.063045e-18,1.132881e-17,-4.531523e-16,1.087565e-16,1.064908e-16,1.631348e-16,1.812609e-17,1.110223e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


Define function to create model. I start off with 2 layers of 8 and 4 neurons each, and a learning rate of 0.01.

In [132]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [133]:
def create_model():
    
    # Create the model.
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model.
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

model = create_model()
print(model.summary())

Model: "sequential_116"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_335 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_336 (Dense)            (None, 4)                 36        
_________________________________________________________________
dense_337 (Dense)            (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


Tune batch size and training epochs. The best model (based on highest accuracy score) has batch size of 10 and training epochs of 50. 

In [135]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold

In [136]:
seed = 9876
np.random.seed(seed)

def create_model():
    
    # Create the model.
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model.
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

model = KerasClassifier(build_fn = create_model, verbose = 1)

batch_size = [10, 40]
epochs = [10, 50]

param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardised, Y)

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

C:\Users\Desiree\Anaconda3\envs\deeplearnproject\lib\site-packages\sklearn\model_selection\_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] batch_size=10, epochs=10 ........................................
Epoch 1/10
261/261 [==============================] - 0s 92us/step - loss: 0.6117 - accuracy: 0.7318
Epoch 2/10
261/261 [==============================] - 0s 107us/step - loss: 0.4378 - accuracy: 0.7854
Epoch 3/10
261/261 [==============================] - 0s 168us/step - loss: 0.4006 - accuracy: 0.8084
Epoch 4/10
261/261 [==============================] - 0s 168us/step - loss: 0.3855 - accuracy: 0.8084
Epoch 5/10
261/261 [==============================] - 0s 165us/step - loss: 0.3722 - accuracy: 0.8276
Epoch 6/10
261/261 [==============================] - 0s 153us/step - loss: 0.3614 - accuracy: 0.8391
Epoch 7/10
261/261 [==============================] - 0s 161us/step - loss: 0.3589 - accuracy: 0.8429
Epoch 8/10
261/261 [==============================] - 0s 172us/step - loss: 0.3565 - accuracy: 0.8391
Epoch 9/10
261/261 [==============================] - 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 0s 88us/step - loss: 0.6262 - accuracy: 0.7203
Epoch 2/10
261/261 [==============================] - 0s 88us/step - loss: 0.4723 - accuracy: 0.7816
Epoch 3/10
261/261 [==============================] - 0s 161us/step - loss: 0.4582 - accuracy: 0.7778
Epoch 4/10
261/261 [==============================] - 0s 130us/step - loss: 0.4432 - accuracy: 0.8008
Epoch 5/10
261/261 [==============================] - 0s 176us/step - loss: 0.4347 - accuracy: 0.7969
Epoch 6/10
261/261 [==============================] - 0s 176us/step - loss: 0.4340 - accuracy: 0.8046
Epoch 7/10
261/261 [==============================] - 0s 142us/step - loss: 0.4282 - accuracy: 0.8046
Epoch 8/10
261/261 [==============================] - 0s 146us/step - loss: 0.4169 - accuracy: 0.8084
Epoch 9/10
261/261 [==============================] - 0s 180us/step - loss: 0.4217 - accuracy: 0.8238
Epoch 10/10
131/131 [==============================] - 0s 31us/step
[CV] ...........

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.5s remaining:    0.0s


...

Epoch 1/10
262/262 [==============================] - 0s 53us/step - loss: 0.6877 - accuracy: 0.6260
Epoch 2/10
262/262 [==============================] - 0s 88us/step - loss: 0.6668 - accuracy: 0.6527
Epoch 3/10
262/262 [==============================] - 0s 80us/step - loss: 0.6172 - accuracy: 0.6985
Epoch 4/10
262/262 [==============================] - 0s 160us/step - loss: 0.5451 - accuracy: 0.7672
Epoch 5/10
262/262 [==============================] - 0s 114us/step - loss: 0.4848 - accuracy: 0.7443
Epoch 6/10
262/262 [==============================] - 0s 65us/step - loss: 0.4760 - accuracy: 0.7328
Epoch 7/10
262/262 [==============================] - 0s 107us/step - loss: 0.4756 - accuracy: 0.7443
Epoch 8/10
262/262 [==============================] - 0s 84us/step - loss: 0.4676 - accuracy: 0.7519
Epoch 9/10
262/262 [==============================] - 0s 76us/step - loss: 0.4619 - accuracy: 0.7634
Epoch 10/10
130/130 [==============================] - 0s 38us/step
[CV] ............ ba

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   31.7s remaining:    0.0s


Epoch 1/50
261/261 [==============================] - 0s 46us/step - loss: 0.6688 - accuracy: 0.6667
Epoch 2/50
261/261 [==============================] - 0s 80us/step - loss: 0.6006 - accuracy: 0.6973
Epoch 3/50
261/261 [==============================] - 0s 123us/step - loss: 0.5163 - accuracy: 0.6973
Epoch 4/50
261/261 [==============================] - 0s 92us/step - loss: 0.4674 - accuracy: 0.7548
Epoch 5/50
261/261 [==============================] - 0s 92us/step - loss: 0.4324 - accuracy: 0.8008
Epoch 6/50
261/261 [==============================] - 0s 119us/step - loss: 0.4072 - accuracy: 0.7969
Epoch 7/50
261/261 [==============================] - 0s 92us/step - loss: 0.3922 - accuracy: 0.8123
Epoch 8/50
261/261 [==============================] - 0s 103us/step - loss: 0.3853 - accuracy: 0.8161
Epoch 9/50
261/261 [==============================] - 0s 134us/step - loss: 0.3767 - accuracy: 0.8199
Epoch 10/50
261/261 [==============================] - 0s 96us/step - loss: 0.3724 - ac

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   45.1s finished


Epoch 1/50
392/392 [==============================] - 0s 74us/step - loss: 0.5579 - accuracy: 0.7321
Epoch 2/50
392/392 [==============================] - 0s 138us/step - loss: 0.4567 - accuracy: 0.7832
Epoch 3/50
392/392 [==============================] - 0s 130us/step - loss: 0.4377 - accuracy: 0.7908
Epoch 4/50
392/392 [==============================] - 0s 176us/step - loss: 0.4351 - accuracy: 0.7832
Epoch 5/50
392/392 [==============================] - 0s 176us/step - loss: 0.4294 - accuracy: 0.7806
Epoch 6/50
392/392 [==============================] - 0s 130us/step - loss: 0.4304 - accuracy: 0.8061
Epoch 7/50
392/392 [==============================] - 0s 148us/step - loss: 0.4254 - accuracy: 0.7934
Epoch 8/50
392/392 [==============================] - 0s 115us/step - loss: 0.4351 - accuracy: 0.7857
Epoch 9/50
392/392 [==============================] - 0s 99us/step - loss: 0.4098 - accuracy: 0.8010
Epoch 10/50
392/392 [==============================] - 0s 161us/step - loss: 0.4083 

Tune learning rate and dropout rate. Hard code the batch size and training epoch obtained previously. The best model has learning rate of 0.01 and dropout rate of 0.1.

In [137]:
from keras.layers import Dropout

In [138]:
seed = 9876
np.random.seed(seed)

def create_model(learn_rate, dropout_rate):
    
    # Create the model.
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate)) 
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model.
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

model = KerasClassifier(build_fn = create_model, batch_size = 10, epochs = 50, verbose = 0)

learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardised, Y)

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

C:\Users\Desiree\Anaconda3\envs\deeplearnproject\lib\site-packages\sklearn\model_selection\_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.733, total=   3.7s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.6s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.763, total=   3.3s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.8s remaining:    0.0s


...

[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.831, total=   2.9s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   29.6s remaining:    0.0s


[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.725, total=   3.5s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.763, total=   4.0s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.815, total=   3.4s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.733, total=   3.6s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.809, total=   3.4s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.838, total=   3.9s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.1, learn_rate=0.1, score=0.733, total=   3.5s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.6min finished
C:\Users\Desiree\Anaconda3\envs\deeplearnproject\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.7933673416169322, using {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7678571402722475 (0.030710750122669134) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7857142844978644 (0.032344602514417495) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7780612243377433 (0.0402725879136073) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.7678571370791416 (0.03693545277935656) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.7933673416169322 (0.04452698200410441) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7525510304436391 (0.05726352155661545) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.7576530635052797 (0.041729797560808295) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7704081704117813 (0.03229035213501209) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.7270408099403187 (0.02004392236020585) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


Tune activation function and network weight initialisation. Hard code the batch size, training epoch and learning rate that were obtained previously. The best model uses softmax activation and uses uniform network weight initialisation.

In [139]:
seed = 9876
np.random.seed(seed)

def create_model(activation, init):
    
    # Create the model.
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer=init, activation=activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer=init, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model.
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

model = KerasClassifier(build_fn = create_model, batch_size = 10, epochs = 50, verbose = 0)

activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

param_grid = dict(activation=activation, init=init)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardised, Y)

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

C:\Users\Desiree\Anaconda3\envs\deeplearnproject\lib\site-packages\sklearn\model_selection\_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=softmax, init=uniform ................................
[CV] .... activation=softmax, init=uniform, score=0.740, total=   2.1s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV] .... activation=softmax, init=uniform, score=0.779, total=   2.5s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.6s remaining:    0.0s


...

[CV] ....... activation=softmax, init=zero, score=0.700, total=   2.1s
[CV] activation=relu, init=uniform ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   22.5s remaining:    0.0s


[CV] ....... activation=relu, init=uniform, score=0.725, total=   2.9s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.779, total=   3.1s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.808, total=   2.5s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.718, total=   2.6s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.779, total=   2.9s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.831, total=   2.9s
[CV] activation=relu, init=zero ......................................
[CV] .......... activation=relu, init=zero, score=0.611, total=   3.0s
[CV] activation=relu, init=zero ......................................
[CV] .

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  1.5min finished
C:\Users\Desiree\Anaconda3\envs\deeplearnproject\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.7959183624812535, using {'activation': 'softmax', 'init': 'uniform'}
0.7959183624812535 (0.05394679691822502) with: {'activation': 'softmax', 'init': 'uniform'}
0.7857142720295458 (0.05659710545962977) with: {'activation': 'softmax', 'init': 'normal'}
0.6683673420730902 (0.04092231924913523) with: {'activation': 'softmax', 'init': 'zero'}
0.7704081536859883 (0.034159148828236695) with: {'activation': 'relu', 'init': 'uniform'}
0.7755102109240026 (0.04624149261633471) with: {'activation': 'relu', 'init': 'normal'}
0.6683673420730902 (0.04092231924913523) with: {'activation': 'relu', 'init': 'zero'}
0.7908163082842924 (0.03370616130132414) with: {'activation': 'tanh', 'init': 'uniform'}
0.7755101941982094 (0.03478923012909107) with: {'activation': 'tanh', 'init': 'normal'}
0.6683673420730902 (0.04092231924913523) with: {'activation': 'tanh', 'init': 'zero'}
0.793367321698033 (0.03176447518730299) with: {'activation': 'linear', 'init': 'uniform'}
0.7857142844978644 (0.031736834261

Tune the number of neurons in the hidden layers. Hard code the batch size, training epoch, learning rate, activation function and network weight initialisation that were obtained previously. The best model has 8 neurons in the 1st layer and 6 neurons in the 2nd layer.

In [140]:
seed = 9876
np.random.seed(seed)

def create_model(neuron1, neuron2):
    
    # Create the model.
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer='uniform', activation='softmax'))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer='uniform', activation='softmax'))
    model.add(Dense(1, activation='sigmoid')) # sigmoid is used in binary classification
    
    # Compile the model.
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

model = KerasClassifier(build_fn = create_model, batch_size = 10, epochs = 50, verbose = 0)

neuron1 = [4, 8, 12]
neuron2 = [2, 4, 6]

param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), refit = True, verbose = 10)
grid_results = grid.fit(X_standardised, Y)

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

C:\Users\Desiree\Anaconda3\envs\deeplearnproject\lib\site-packages\sklearn\model_selection\_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=4, neuron2=2 ............................................
[CV] ................ neuron1=4, neuron2=2, score=0.702, total=   2.3s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=2, score=0.779, total=   3.0s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.2s remaining:    0.0s


...

[CV] ................ neuron1=4, neuron2=6, score=0.823, total=   2.5s
[CV] neuron1=8, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   23.3s remaining:    0.0s


[CV] ................ neuron1=8, neuron2=2, score=0.718, total=   2.4s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.786, total=   2.4s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.846, total=   3.3s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.740, total=   2.3s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.779, total=   3.0s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.862, total=   2.1s
[CV] neuron1=8, neuron2=6 ............................................
[CV] ................ neuron1=8, neuron2=6, score=0.779, total=   2.6s
[CV] neuron1=8, neuron2=6 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.1min finished


Best: 0.8010204017770534, using {'neuron1': 8, 'neuron2': 6}
0.7729591745503095 (0.055700763011101016) with: {'neuron1': 4, 'neuron2': 2}
0.7780612138461094 (0.02431947857888487) with: {'neuron1': 4, 'neuron2': 4}
0.7831632742772296 (0.03759812018049345) with: {'neuron1': 4, 'neuron2': 6}
0.7831632648499645 (0.052511303001968164) with: {'neuron1': 8, 'neuron2': 2}
0.7933673521085661 (0.05049090032359488) with: {'neuron1': 8, 'neuron2': 4}
0.8010204017770534 (0.026557574151598942) with: {'neuron1': 8, 'neuron2': 6}
0.7857142677720712 (0.02650267600187418) with: {'neuron1': 12, 'neuron2': 2}
0.7857142782637051 (0.044365707126585066) with: {'neuron1': 12, 'neuron2': 4}
0.7857142981826043 (0.046508593883445554) with: {'neuron1': 12, 'neuron2': 6}


Use the best model with tuned parameters to make predictions.

In [142]:
y_pred = grid.predict(X_standardised)
print(y_pred[:7].reshape(1, -1))

[[0 1 0 1 1 1 1]]


Calculate model performance metrics. The best model predicted that 262 persons do not have diabetes and 130 persons have diabetes. 

In [143]:
from sklearn.metrics import classification_report, accuracy_score

In [144]:
print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.8698979591836735
              precision    recall  f1-score   support

           0       0.92      0.88      0.90       262
           1       0.78      0.85      0.81       130

    accuracy                           0.87       392
   macro avg       0.85      0.87      0.86       392
weighted avg       0.87      0.87      0.87       392



The 3rd data point is actually class 1, but is predicted as class 0 by the model. This is a false negative.

In [145]:
example = df.iloc[2]
print(example)

n_pregnant                  3.000
glucose_concentration      78.000
blood_pressure (mm Hg)     50.000
skin_thickness (mm)        32.000
serum_insulin (mu U/ml)    88.000
BMI                        31.000
pedigree_function           0.248
age                        26.000
class                       1.000
Name: 6, dtype: float64


In [146]:
prediction = grid.predict(X_standardised[2].reshape(1, -1))
print(prediction)

[[0]]
